In [1]:
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import hist
from hist import Hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
NanoAODSchema.warn_missing_crossrefs = False

from coffea.analysis_tools import PackedSelection
from topcoffea.modules import utils
import topcoffea.modules.eft_helper as efth

In [2]:
# histogram style
hep.style.use("CMS")
params = {'axes.labelsize': 20,
          'axes.titlesize': 20,
          'legend.fontsize':20}
plt.rcParams.update(params)

In [4]:
fname = '/project01/ndcms/hnelson2/ttbarEFT/nanoGen/TT01j2l_S1/nanoGen_101.root'

In [5]:
# Load in events from root file
events = NanoEventsFactory.from_root(
    fname,
    schemaclass=NanoAODSchema.v6,
    metadata={"dataset": "TT01j2l"},
).events()

In [7]:
wc_lst = utils.get_list_of_wc_names(fname)
print(wc_lst)

['cQd1', 'ctj1', 'cQj31', 'ctj8', 'ctd1', 'ctd8', 'ctGRe', 'ctGIm', 'cQj11', 'cQj18', 'ctu8', 'cQd8', 'ctu1', 'cQu1', 'cQj38', 'cQu8']


In [8]:
events.fields

['MET',
 'GenIsolatedPhoton',
 'LHEScaleWeight',
 'LHEPart',
 'luminosityBlock',
 'GenJetAK8',
 'GenVisTau',
 'LHE',
 'GenDressedLepton',
 'GenVtx',
 'GenJet',
 'genWeight',
 'run',
 'EFTfitCoefficients',
 'GenMET',
 'Generator',
 'PSWeight',
 'LHEWeight',
 'event',
 'WCnames',
 'LHEPdfWeight',
 'LHEReweightingWeight',
 'GenPart']

In [9]:
events.GenJet.fields

['eta', 'mass', 'phi', 'pt', 'partonFlavour', 'hadronFlavour']

In [32]:
events.LHEPart.fields

['pt', 'eta', 'phi', 'mass', 'incomingpz', 'pdgId', 'status', 'spin']

In [31]:
events.GenPart.fields

['eta',
 'mass',
 'phi',
 'pt',
 'genPartIdxMother',
 'pdgId',
 'status',
 'statusFlags',
 'genPartIdxMotherG',
 'distinctParentIdxG',
 'childrenIdxG',
 'distinctChildrenIdxG',
 'distinctChildrenDeepIdxG']